In [1]:
import praw

In [3]:
# client_id='PERSONAL_USE_SCRIPT_14_CHARS', 
#                      client_secret='SECRET_KEY_27_CHARS ', 
#                      user_agent='YOUR_APP_NAME'
reddit = praw.Reddit(client_id='R30fcxAZLJ7Wyw',
                     client_secret='8Zf87krHYa_zXRXPWEF8hK2Twfs',
                     user_agent='project1')

In [4]:
subreddit = reddit.subreddit('india')

In [5]:
# Each subreddit has five different ways of organizing the topics created by redditors: .hot, .new,
# .controversial, .top, and .gilded. 
# You can also use .search("SEARCH_KEYWORDS") to get only results matching an engine search.

In [6]:
# get 10 hot posts from the MachineLearning subreddit
hot_posts = subreddit.hot(limit=10)

In [9]:
for post in hot_posts:
    print(post.title)

Coronavirus (COVID-19) Megathread - News and Updates - 4
Announcing r/IndiaMeme, our own sub for memes that are related to India. Welcoming all the memers to submit their memes and OC memes!
Pizza delivery boy tests positive, 72 families in South Delhi ordered to quarantine themselves
After protests, Kerala local BJP leader arrested for allegedly raping his student
Tens of thousands of Chinese PPE kits fail India safety test
Dads, Indian Dads
Twitter suspends (Kangana Ranaut's sister) Rangoli Chandel's account after controversial posts.
India charges Tablighi Jammat chief with culpable homicide for coronavirus surge
400 families block Bengal highway for 3 hours alleging no food amid lockdown
E-commerce companies allowed to deliver essential and non-essential items from April 20
